In [19]:
from collections import defaultdict
from datetime import datetime
import torch
import numpy as np
import pandas as pd
from datetime import datetime

In [20]:
ml_1m_path = '../../../datasets/ml-1m'
PATH_TO_PROCESSED_DATA = 'processed/'
dataAfter = 'preprocessed_data/'
dayTime = 86400

In [21]:
file_path = ml_1m_path + '/ratings.dat'
data_ratings = pd.read_csv(file_path, sep='::', engine='python',
                   names=['SessionID', 'ItemID', 'Rating', 'Time'], index_col='SessionID').drop('Rating', axis=1, inplace=False)

## SASRec handling
Only use implicit feedback in the sequence of items

In [22]:
data_ratings

,ItemID,Time
SessionID,,
1,1193,978300760
1,661,978302109
1,914,978301968
1,3408,978300275
1,2355,978824291
...,...,...
6040,1091,956716541
6040,1094,956704887
6040,562,956704746


In [23]:
actions = data_ratings
actions = actions.groupby('ItemID').filter(lambda  x: len(x) >= 5)
actions = actions.groupby('SessionID').filter(lambda  x: len(x) >= 5)
actions = actions.groupby('SessionID', group_keys=False).apply(lambda  x: x.sort_values('Time'))
# We discard users and items with fewer than 5 related actions.
actions

,ItemID,Time
SessionID,,
1,3186,978300019
1,1721,978300055
1,1022,978300055
1,1270,978300055
1,2340,978300103
...,...,...
6040,2917,997454429
6040,1784,997454464
6040,1921,997454464


In [24]:
cols = actions.columns.tolist()  # get a list of all the columns
b, c = cols.index('ItemID'), cols.index('Time')  # get the index of columns 'B' and 'C'
cols[b], cols[c] = cols[c], cols[b]  # swap the columns
actions = actions[cols]
# swap the Time and ItemID columns

In [25]:
usermap = dict()
usernum = 0
itemmap = dict()
itemnum = 0
# reorder the userid and itemid (keep the same step with original SASRec code)
for _id, row in actions.iterrows():
    if _id in usermap:
        userid = usermap[_id]
    else:
        usernum += 1
        userid = usernum
        usermap[_id] = userid

    if row.ItemID in itemmap:
        itemid = itemmap[row.ItemID]
    else:
        itemnum += 1
        itemid = itemnum
        itemmap[row.ItemID] = itemid

In [26]:
actions['ItemID'] = actions['ItemID'].map(itemmap)
actions.index = actions.index.map(usermap)
# data['Occupation'] = data['Occupation'].map(occupation_map)

In [27]:
actions.index.nunique()

6040

In [28]:
Train = actions.groupby('SessionID', group_keys=False).apply(lambda x: x.iloc[:-2])
Valid = actions.groupby('SessionID', group_keys=False).apply(lambda x: x.iloc[:-1])
Test = actions
# Test = actions.groupby('SessionID', group_keys=False)

In [29]:
actions

,Time,ItemID
SessionID,,
1,978300019,1
1,978300055,2
1,978300055,3
1,978300055,4
1,978300103,5
...,...,...
6040,997454429,1249
6040,997454464,88
6040,997454464,371


In [30]:
Train.to_csv(dataAfter + 'ml-1m-Train_SAS.txt', sep=',', index=True)
Valid.to_csv(dataAfter + 'ml-1m-Valid_SAS.txt', sep=',', index=True)
Test.to_csv(dataAfter + 'ml-1m-Test_SAS.txt', sep=',', index=True)